<a href="https://colab.research.google.com/github/derzhavin3016/CompMath/blob/master/Lab2/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя лабораторная работа №2 по вычислительной математике
Державин Андрей, Б01-909 группа


Задача II.10.6 (к)

$$
  \left\lbrace
  \begin{matrix}
    a_{11} x_1 + a_{12} x_2 + \dots + a_{1n} x_n &= &f_1 \\
    \dots  &\dots   &\dots \\
    a_{n1} x_1 + a_{n2} x_2 + \dots + a_{nn} x_n &= &f_n
  \end{matrix}
  \right. ,
$$
где
$$
n = 10, \: 
a_{ii} = 1, \: 
a_{ij} = \frac{1}{i + j} (i \neq j), \:
f_i = \frac{1}{i} 
$$

In [115]:
import numpy as np
from numpy import linalg as ling

## Определяем константы: размер матрицы и погрешность (для сравнения чисел с 0)

In [116]:
SIZE = 10
ACCURACY = 1e-10

# Метод Гаусса

## Функция сравнения числа с нулём с учётом погрешности

In [117]:
def is_zero(num: float):
  return abs(num) < ACCURACY

## Функция нахождения срочки в матрице, в которой в столбце `idx` стоит ненулеове число

In [118]:
def find_non_zero(arr, col_idx):
  for i in range(col_idx + 1, SIZE):
    if not is_zero(arr[i][col_idx]):
      return col_idx
  return -1

## Функция приведения к диагональному виду (прямой ход Гаусса)

In [119]:
def to_diag(matr):
  for i in range(SIZE - 1):
    if is_zero(matr[i][i]):
      non_z_line = find_non_zero(matr, i)
      if non_z_line == -1:
        raise ValueError("Invalid Matrix")
      # swap rows
      matr[[i, non_z_line]] = matr[[non_z_line, i]]

    div = matr[i][i]
    for j in range(i + 1, SIZE):
      matr[j] += (-matr[j][i] / div) * matr[i] 

## Вспомогательные функции:
1. `zero_small` - замена всех малых чисел на 0
1. `arr_print` - распечатать матрицу в красивом виде

In [120]:
def zero_small(matr):
  for i in range(matr.shape[0]):
    for j in range(matr.shape[1]):
      if is_zero(matr[i][j]):
        matr[i][j] = 0
def arr_print(matr):
  for row in matr:
    for elem in row:
      print(f"{elem} ", end="")
    print()


## Функция обратного хода Гаусса, возвращает столбец решений

In [121]:
def rev_hod(matr):
  sz = matr.shape[0]
  sol = np.zeros((sz, 1))
  sol[sz - 1][0] = matr[sz - 1][sz] / matr[sz - 1][sz - 1]
  for i in range(sz - 2, -1, -1):
    sum = 0
    for j in range(i + 1, sz):
      sum += matr[i][j] * sol[j][0]
    sol[i][0] = (matr[i][sz] - sum) / matr[i][i]
  return sol

## Основная программа:
1. Заполенение матрицы $A$ и столбца $f$
1. "Приклеивание" к матрице столбца
1. Выполнение прямого и обратного ходов
1. Печать вектора решений

In [122]:
A = np.ones((SIZE, SIZE))
for i in range(SIZE):
  for j in range(SIZE):
    if i != j:
      A[i][j] = 1 / (i + j + 2)
f = np.array([[1 / i for i in range(1, SIZE + 1)]]).transpose()
Asys = np.hstack((A, f))
to_diag(Asys)
#zero_small(Asys) # set all small numbers to zero (< 1e-10)
#arr_print(Asys)
sol = rev_hod(Asys)
print(f"Решение x = \n{sol}")

Решение x = 
[[ 9.19077109e-01]
 [ 1.75540170e-01]
 [ 6.39348240e-02]
 [ 2.72747640e-02]
 [ 1.14234685e-02]
 [ 3.51083928e-03]
 [-7.89957814e-04]
 [-3.25080145e-03]
 [-4.69787781e-03]
 [-5.55373994e-03]]


In [123]:
def vec_n1(vec):
  return max(abs(vec))

def vec_n2(vec):
  return sum(abs(vec))

def vec_n3(vec):
  return np.sqrt(sum(vec * vec))

## Вывод  невязки.

In [124]:
diff  = (np.linalg.solve(A, f) - sol)
err = f - np.dot(A, sol)

print(f'Невязка 1: {vec_n1(err)[0]}')
print(f'Невязка 2: {vec_n2(err)[0]}')
print(f'Невязка 3: {vec_n3(err)[0]}')

Невязка 1: 2.7755575615628914e-17
Невязка 2: 1.1102230246251565e-16
Невязка 3: 5.1925927263190304e-17


## Функции для подсчёта норм

In [125]:
def norm_1(matr):
  max_s = 0
  rows, cols = matr.shape
  for j in range(cols):
    sum = 0
    for i in range(rows):
      sum += abs(matr[i][j])
    max_s = max(sum, max_s)

  return max_s

def norm_2(matr):
  rows, cols = matr.shape
  max_s = 0
  for i in range(rows):
    max_s = max(sum(abs(matr[i])), max_s)

  return max_s

def norm_3(matr):
  return np.sqrt(max(abs(np.linalg.eigvals(np.dot(matr, matr.transpose())))))

## Подсчёт обсуловленности через 3 разные нормы

In [126]:
Ainv = np.linalg.inv(A)
mus = [
    norm_1(A) * norm_1(Ainv),
    norm_2(A) * norm_2(Ainv),
    norm_3(A) * norm_3(Ainv)
]
for i, elem in enumerate(mus):
  print(f"mu_{i} = {elem}")

mu_0 = 5.6336089382284005
mu_1 = 5.6336089382284005
mu_2 = 3.113199424608562


# Метод Зейделя

In [141]:
def get_LUD(matr):
  sz = matr.shape[0]
  D = np.zeros((sz, sz))
  L = np.zeros((sz, sz))
  U = np.zeros((sz, sz))
  for i in range(sz):
    for j in range(sz):
      if i == j:
        D[i][j] = matr[i][j]
      elif i > j:
        L[i][j] = matr[i][j]
      else:
        U[i][j] = matr[i][j]
  return L, U, D

In [142]:
def zeidel(matr, b, eps, norm):
  print(f"Норма {norm.__name__}")
  it = 0
  converged = False
  
  x = np.zeros((SIZE, 1))
  
  L, U, D = get_LUD(matr)
  LDinv = np.linalg.inv(L + D)
  LDinvU = np.dot(LDinv, U)
  
  while not converged:
    it += 1
    x_new = -np.dot(LDinvU, x) + np.dot(LDinv, b)
    converged = norm(x_new - x) < eps
    x = x_new
  
  print(f'Количество итераций {it}')
  print(f'Невязка: {norm(b - np.dot(matr,x))[0]}')
  print(f'Условие останова ||x_{{k+1}} - x_k||_{norm.__name__[-1]} < {eps}')
  print(f'Решение x = \n{x}')

In [151]:
eps = 1e-16

In [152]:
zeidel(A, f, eps, vec_n1)

Норма vec_n1
Количество итераций 25
Невязка: 5.551115123125783e-17
Условие останова ||x_{k+1} - x_k||_1 < 1e-16
Решение x = 
[[ 9.19077109e-01]
 [ 1.75540170e-01]
 [ 6.39348240e-02]
 [ 2.72747640e-02]
 [ 1.14234685e-02]
 [ 3.51083928e-03]
 [-7.89957814e-04]
 [-3.25080145e-03]
 [-4.69787781e-03]
 [-5.55373994e-03]]


In [153]:
zeidel(A, f, eps, vec_n2)

Норма vec_n2
Количество итераций 25
Невязка: 2.220446049250313e-16
Условие останова ||x_{k+1} - x_k||_2 < 1e-16
Решение x = 
[[ 9.19077109e-01]
 [ 1.75540170e-01]
 [ 6.39348240e-02]
 [ 2.72747640e-02]
 [ 1.14234685e-02]
 [ 3.51083928e-03]
 [-7.89957814e-04]
 [-3.25080145e-03]
 [-4.69787781e-03]
 [-5.55373994e-03]]


In [154]:
zeidel(A, f, eps, vec_n3)

Норма vec_n3
Количество итераций 25
Невязка: 9.614813431917819e-17
Условие останова ||x_{k+1} - x_k||_3 < 1e-16
Решение x = 
[[ 9.19077109e-01]
 [ 1.75540170e-01]
 [ 6.39348240e-02]
 [ 2.72747640e-02]
 [ 1.14234685e-02]
 [ 3.51083928e-03]
 [-7.89957814e-04]
 [-3.25080145e-03]
 [-4.69787781e-03]
 [-5.55373994e-03]]
